In [1]:
client_id = ''
client_secret = ''

import pandas as pd
from datetime import datetime, timedelta
import requests
import re
def handle_authorization(data):
    
    access_token = data["access_token"]
    print(f" {access_token}")
    return access_token
    
def get_twitch_authorization(client_id, client_secret):
    url = f"https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
    }
    
    response = requests.post(url, params=params)
    data = response.json()
    
    handle_authorization(data)



# Obtén la autorización de Twitch
get_twitch_authorization(client_id, client_secret)

 pc32eqtbzk347c80p3j489jc05suaz


In [2]:
oauth_token = ''

In [7]:
def extract_channel_name(url):
    pattern = r"https?:\/\/(?:www\.|go\.)?twitch\.tv\/([a-zA-Z0-9_]{4,25})"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None
def get_streamer_data(client_id, url):
    channel_name = extract_channel_name(url)
    if not channel_name:
        return None
    
    url = f"https://api.twitch.tv/helix/users"
    
    params = {
        "login": channel_name
    }
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }
    
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    print(data)
    if "data" in data and len(data["data"]) > 0:
        user_id = data["data"][0]["id"]
        stream_url = f"https://api.twitch.tv/helix/streams?user_id={user_id}"
        response = requests.get(stream_url, headers=headers)
        stream_data = response.json()
        
        if "data" in stream_data and len(stream_data["data"]) > 0:
            return stream_data["data"][0]
    
    return None
url = "https://www.twitch.tv/adricontreras4"
streamer_data = get_streamer_data(client_id, url)

if streamer_data:
    data = f"Datos del streamer {channel_url}:"

{'data': [{'id': '513284706', 'login': 'adricontreras4', 'display_name': 'AdriContreras4', 'type': '', 'broadcaster_type': 'partner', 'description': 'Canal personal de Adri Contreras, periodista deportivo y creador de contenido.', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/743f0bf4-6370-4aa6-adba-a90933e63687-profile_image-300x300.png', 'offline_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/07f01fc0-8042-4681-8388-a9f88c82aaff-channel_offline_image-1920x1080.png', 'view_count': 0, 'created_at': '2020-04-13T01:10:20Z'}]}


In [8]:
data = {'data': [{'id': '513284706', 'login': 'adricontreras4', 'display_name': 'AdriContreras4', 'type': '', 'broadcaster_type': 'partner', 'description': 'Canal personal de Adri Contreras, periodista deportivo y creador de contenido.', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/743f0bf4-6370-4aa6-adba-a90933e63687-profile_image-300x300.png', 'offline_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/07f01fc0-8042-4681-8388-a9f88c82aaff-channel_offline_image-1920x1080.png', 'view_count': 0, 'created_at': '2020-04-13T01:10:20Z'}]}
data_dict = data['data'][0]
id_canal =  list(data_dict.values())[0]
id_canal

'513284706'

In [9]:
def obtener_vods(id_canal):
    # Calcular la fecha hace un año desde hoy
    fecha_inicio = datetime.now() - timedelta(days=730)
    fecha_inicio_str = fecha_inicio.strftime("%Y-%m-%dT%H:%M:%SZ")

    url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100"

    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }

    vods_data = []

    while url:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            vods = data["data"]

            for vod in vods:
                vod_id = int(vod["id"])
                fecha_inicio = datetime.strptime(vod["created_at"], "%Y-%m-%dT%H:%M:%SZ")
                fecha_final = datetime.strptime(vod["published_at"], "%Y-%m-%dT%H:%M:%SZ")
                duracion = vod["duration"]
                visitas = int(vod["view_count"])
                nombre = vod["title"].lower().replace(' ', '_')
                descripcion = vod["description"]
                url_vod = f"https://www.twitch.tv/videos/{vod_id}"

                vods_data.append({
                    "ID del VOD": vod_id,
                    "URL del VOD": url_vod,
                    "Nombre del VOD": nombre,
                    "Duración": duracion,
                    "Fecha de inicio": fecha_inicio,
                    "Fecha de finalización": fecha_final,
                    "Visitas": visitas,
                    
                })

            pagination = data.get("pagination", None)
            if pagination and "cursor" in pagination:
                cursor = pagination["cursor"]
                url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100&after={cursor}"
            else:
                url = None

        else:
            print("Se produjo un error al realizar la solicitud GET.")
            return None

    # Crear un DataFrame a partir de los datos
    df_vods = pd.DataFrame(vods_data)

    return df_vods

df = obtener_vods(id_canal)
df.info(memory_usage='deep')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID del VOD             28 non-null     int64         
 1   URL del VOD            28 non-null     object        
 2   Nombre del VOD         28 non-null     object        
 3   Duración               28 non-null     object        
 4   Fecha de inicio        28 non-null     datetime64[ns]
 5   Fecha de finalización  28 non-null     datetime64[ns]
 6   Visitas                28 non-null     int64         
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 8.3 KB


,ID del VOD,URL del VOD,Nombre del VOD,Duración,Fecha de inicio,Fecha de finalización,Visitas
0,1886555722,https://www.twitch.tv/videos/1886555722,anuncio_los_descartes_y_4_jugadores_que_siguen...,4h43m2s,2023-07-31 16:37:57,2023-07-31 16:37:57,74812
1,1884792848,https://www.twitch.tv/videos/1884792848,final_four_kings_&_queens_league,7h3m29s,2023-07-29 14:40:28,2023-07-29 14:40:28,405046
2,1883964280,https://www.twitch.tv/videos/1883964280,meet_&_greet_pre_final_four,3h28m22s,2023-07-28 15:58:50,2023-07-28 15:58:50,31111
3,1882376735,https://www.twitch.tv/videos/1882376735,el_barrio_jugará_la_final_four_en_el_metropoli...,2h32m1s,2023-07-26 19:04:58,2023-07-26 19:04:58,13777
4,1881506700,https://www.twitch.tv/videos/1881506700,el_barrio_vs_troncas_|_cuartos_de_final_queens...,59m24s,2023-07-25 19:15:02,2023-07-25 19:15:02,38563


In [10]:
df.to_csv('twitch_api_adricontreras4_csv.csv', index=False)